# Imports 

In [1]:
import requests 
import read_write_qrcd as q_reader
import pysolr

# define functions 

In [2]:
def del_solr_data(solr_core='qa_core', url='http://localhost:8983/solr/'):
    import requests
    headers = { 'Content-Type': 'text/xml'}
    params = { 'commit': 'true'}
    data = '<delete><query>*:*</query></delete>'

    response = requests.post(
        '{}{}/update'.format(url, solr_core), 
        headers=headers, 
        params=params, 
        data=data)
    return response
################################################
def prepare_data(data_item):
    result_list = []
    
    pq_id = data_item['pq_id'].split(':')[0]
    verse_start = int(data_item['verses'].split('-')[0])
    passages = [ver.strip() for ver in data_item.get('passage').split('.') if ver.strip()]
    for passage in passages:
        doc = {
            'id': pq_id + '_' + str(verse_start),
            'doc': passage
        }
        verse_start+=1
        result_list.append(doc)

    return result_list
################################################

def solr_ir(docs_data, query, solr_engine):
    # make_sure there is no indexed docs 
    solr_engine.delete(q='*:*')
    # index docs 
    solr_engine.add(docs_data)
    # query 
    query = query.replace('؟', '').replace('"', '')
    search_results = solr_engine.search(
        q=query, 
        fl='id,doc,score', df='doc')
    result_list = []
    for r in search_results:
        #print("id: {}. doc: {}. score: {}".format(r['id'], r['doc'], r['score']))
        result_list.append( {'id': r['id'], 'doc': r['doc'], 'score': r['score'] })
    # delete docs 
    solr_engine.delete(q='*:*')
    # return results 
    return search_results

# Load data 

In [3]:
train_data_file = '../quranqa/datasets/qrcd_v1.1_train.jsonl'
train = q_reader.load_jsonl(train_data_file)

Loaded 710 records from ../quranqa/datasets/qrcd_v1.1_train.jsonl


# Solr URL

In [4]:
solr_url = 'http://localhost:8983/solr/qa_core'

# define a solr client & test connection (ping)

In [5]:
solr = pysolr.Solr(url=solr_url, always_commit=True)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":1,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"-19"}},\n  "status":"OK"}\n'

In [6]:
def run_experiment(data_id):
    print('data instance')
    data_instance = train[data_id]
    print(data_instance)
    print('++++++++++++++++++++++++++++')
    print('prepared data')
    docs_data = prepare_data(data_instance)
    print(docs_data)
    print('++++++++++++++++++++++++++++')
    query = data_instance.get('question')
    print('question:', query)
    print('++++++++++++++++++++++++++++')
    answers = data_instance.get('answers')
    print('answers:', answers)
    print('++++++++++++++++++++++++++++')
    print('Solr query results')
    results = solr_ir(docs_data=docs_data, query=query, solr_engine=solr)
    for r in results:
        print(r)
    

# Result Example 1

In [7]:
run_experiment(0)

data instance
{'pq_id': '2:8-16_364', 'passage': 'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين. يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون. في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون. وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون. ألا إنهم هم المفسدون ولكن لا يشعرون. وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون. وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون. الله يستهزئ بهم ويمدهم في طغيانهم يعمهون. أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين.', 'surah': 2, 'verses': '8-16', 'question': 'لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ""من يضلل الله فما له من هاد"" كما ورد من قوله تعالى في آية 23 و آية 36 من سورة الزمر؟', 'answers': [{'text': 'أولئك الذين اشتروا الضلالة بالهدى', 'start_char': 504}]}
++++++++++++++++++++++++++++
prepared data
[{'id': '2_8', 'doc': 'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بم

# Result Example 2

In [8]:
run_experiment(23)

data instance
{'pq_id': '35:15-18_378', 'passage': 'يا أيها الناس أنتم الفقراء إلى الله والله هو الغني الحميد. إن يشأ يذهبكم ويأت بخلق جديد. وما ذلك على الله بعزيز. ولا تزر وازرة وزر أخرى وإن تدع مثقلة إلى حملها لا يحمل منه شيء ولو كان ذا قربى إنما تنذر الذين يخشون ربهم بالغيب وأقاموا الصلاة ومن تزكى فإنما يتزكى لنفسه وإلى الله المصير.', 'surah': 35, 'verses': '15-18', 'question': 'إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟', 'answers': [{'text': 'من تزكى فإنما يتزكى لنفسه', 'start_char': 242}]}
++++++++++++++++++++++++++++
prepared data
[{'id': '35_15', 'doc': 'يا أيها الناس أنتم الفقراء إلى الله والله هو الغني الحميد'}, {'id': '35_16', 'doc': 'إن يشأ يذهبكم ويأت بخلق جديد'}, {'id': '35_17', 'doc': 'وما ذلك على الله بعزيز'}, {'id': '35_18', 'doc': 'ولا تزر وازرة وزر أخرى وإن تدع مثقلة إلى حملها لا يحمل منه شيء ولو كان ذا قربى إنما تنذر الذين يخشون ربهم بالغيب وأقاموا الصلاة ومن تزكى فإنما يتزكى لنفسه وإلى الله المصير'}]
++++++++++++++++++++++++++++
question: إن كان الله قدّر علي أفعا

# Result Example 3

In [9]:
run_experiment(158)

data instance
{'pq_id': '4:44-46_407', 'passage': 'ألم تر إلى الذين أوتوا نصيبا من الكتاب يشترون الضلالة ويريدون أن تضلوا السبيل. والله أعلم بأعدائكم وكفى بالله وليا وكفى بالله نصيرا. من الذين هادوا يحرفون الكلم عن مواضعه ويقولون سمعنا وعصينا واسمع غير مسمع وراعنا ليا بألسنتهم وطعنا في الدين ولو أنهم قالوا سمعنا وأطعنا واسمع وانظرنا لكان خيرا لهم وأقوم ولكن لعنهم الله بكفرهم فلا يؤمنون إلا قليلا.', 'surah': 4, 'verses': '44-46', 'question': 'هل ذكر القرآن أن التوراة تم تحريفها؟', 'answers': [{'text': 'من الذين هادوا يحرفون الكلم عن مواضعه', 'start_char': 133}]}
++++++++++++++++++++++++++++
prepared data
[{'id': '4_44', 'doc': 'ألم تر إلى الذين أوتوا نصيبا من الكتاب يشترون الضلالة ويريدون أن تضلوا السبيل'}, {'id': '4_45', 'doc': 'والله أعلم بأعدائكم وكفى بالله وليا وكفى بالله نصيرا'}, {'id': '4_46', 'doc': 'من الذين هادوا يحرفون الكلم عن مواضعه ويقولون سمعنا وعصينا واسمع غير مسمع وراعنا ليا بألسنتهم وطعنا في الدين ولو أنهم قالوا سمعنا وأطعنا واسمع وانظرنا لكان خيرا لهم وأقوم ولكن لعنهم 